In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
from pathlib import Path
import matplotlib.pyplot as plt
import csv

In [2]:
#데이터셋
DATASET_NAME = 'kfood'
DRIVE_PATH = Path(os.getcwd())
DATASET_PATH = DRIVE_PATH / DATASET_NAME
filepath = DATASET_PATH
print(filepath.exists())

True


In [12]:
#class_label 매칭 딕셔너리로 저장
labels = None
classes = None
with open(filepath / 'class_label.csv','r') as f:
    w = csv.reader(f)
    classes = w.__next__()
    labels = w.__next__()
#print(len(classes), len(labels), classes, labels)
class_to_label = {}
for _class, _label in zip(classes, labels):
    class_to_label[_class] = int(_label)
tf_class_to_label = tf.constant(list(class_to_label.keys()))

In [13]:
def get_image_crop_points(filepath):
    crops = {}
    properties = filepath / "crop_area.properties"
    with open(properties, 'r') as p:
        for row in p:
            name, crop = row.replace("\n", "").replace(" ", "").split("=")
            if name != "" and crop != "":
                #name = name.encode('utf-8')
                crop = crop.split(",")
                if len(crop) >= 4:
                    crop = [int(crop[1]), int(crop[0]), int(crop[3]), int(crop[2])]
                    crops[name] = crop
                elif len(crop) == 2:
                    crop = [0, 0, int(crop[1]), int(crop[0])]
                    crops[name] = crop
            
    return crops

In [16]:
#crop 지점 정보 빼오기
crop_points = {}
class_list = list(filepath.glob("*/*"))
class_list = [class_name for class_name in class_list if class_name.is_dir()]
for class_name in class_list:
    crop_points.update(get_image_crop_points(class_name))

tf_crop_image_names = tf.constant(list(crop_points.keys()), dtype=tf.string)
tf_crop_points = tf.constant(list(crop_points.values()), dtype=tf.uint8)


In [17]:
#데이터셋의 이미지 경로 및 레이블 저장
from glob import glob
image_paths = sorted(glob("kfood/*/*/*"))
image_paths = [image_path for image_path in image_paths if image_path.split("/")[-1].split(".")[-1].lower() in ("png", "jpg", "jpeg")]
#labels = [class_to_label[Path(image_path).parent.stem] for image_path in image_paths]
print(len(image_paths))#, len(labels))


150507


In [127]:
def parse_and_crop_image_add_label(tf_filepath):
    
    image = tf.io.read_file(tf_filepath) # 이미지 파일 읽기
    filepath = tf.compat.path_to_str(tf_filepath)
    #format decoding
    image_format = tf.strings.split(filepath, ".")[-1]
    if image_format == "jpeg":
        image = tf.image.decode_jpeg(image, channels=3) # JPEG-encoded -> uint8 tensor (RGB format)
    elif image_format == 'png':
        image = tf.image.decode_png(image, channels=3)
    else:
        image = tf.image.decode_image(image, channels=3)
    
    #crop
    image_name = tf.strings.split(tf.strings.split(filepath, "/")[-1], ".")[0]
    tf_image_idx = tf.where(tf_crop_image_names == image_name)
    try:
        image = tf.image.crop_to_bounding_box(image, *tf_crop_points[tf.reshape(tf_image_idx, shape=())])
    except:
        pass
    
    #labeling
    class_name = tf.strings.split(filepath, "/")[-2]
    tf_class_name_idx = tf.where(tf_class_to_label == class_name)
    try:
        label = tf.reshape(tf_class_name_idx, shape=())
    except:
        label = 0
        print("label error")

    return image, int(label)

In [134]:
def make_kfood_dataset(filepaths, n_read_threads=5, shuffle_buffer_size=None, n_parse_threads=5, batch_size=32, cache=False):

    filenames_dataset = tf.data.Dataset.from_tensor_slices(filepaths)
    dataset = filenames_dataset.map(parse_and_crop_image_add_label, num_parallel_calls=n_parse_threads)
    #dataset = filenames_dataset.map(spa)
    if cache:
        dataset = dataset.cache()
    if shuffle_buffer_size:
        dataset = dataset.shuffle(shuffle_buffer_size)
    if batch_size:
        dataset = dataset.batch(batch_size)
    return dataset.prefetch(1)

In [135]:
#데이터셋 생성
dataset = make_kfood_dataset(image_paths, shuffle_buffer_size=10000, n_parse_threads=tf.data.AUTOTUNE, batch_size=None)

In [64]:
crop_points = {}
class_list = list(filepath.glob("*/*"))
print(len(class_list))
class_list = [class_name for class_name in class_list if class_name.is_dir()]
print(len(class_list))
for class_name in class_list:
    crop_points.update(get_image_crop_points(class_name))

150
150


In [65]:
#{ image_filename : crop_point, ... }
len(crop_points.keys())

51660

In [66]:
from PIL import Image
tf.shape(parse_and_crop_image(image_paths[9]))

NameError: name 'parse_and_crop_image' is not defined

In [ ]:
filenames_dataset = tf.data.Dataset.from_tensor_slices(image_paths)
images_ds = filenames_dataset.map(parse_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)


tf.Tensor([], shape=(0,), dtype=int32)
tf.Tensor([400 534   3], shape=(3,), dtype=int32)


In [ ]:
feature_descriptions = {
    "image": tf.io.FixedLenFeature([], tf.string, default_value=""),
    "label": tf.io.FixedLenFeature([], tf.int64, default_value=0)
}
for tfrecord in dataset.take(1):
    example = tf.io.parse_single_example(tfrecord, feature_descriptions)

2021-12-24 13:16:07.492231: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-24 13:16:07.492404: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


KeyboardInterrupt: 